# K-means From Scratch

In this notebook, we will code the k-means model from scratch to understand the theory behind this commonly used unsupervised machine learning model.

In the end, we will compare the result of the k-means model we build from scratch with the one from scikit-learn.

YT: https://youtu.be/uLs-EYUpGAw

## Implementation
The input data is a data frame and "k" (an integer specifying the number of clusters).

In [1]:
# import library
import random

For the first main function, we perform the steps as follows:
- Transform "data" data frame into numpy arrays.
- Initialize the centroids using `initialize_centroids` based on the "data" and the number of clusters ("k").
- Use the next helper function, `get_labels`, to find labels for data points.
- Update the clusters' centroids using the helper function `update_centroids`.
- Repeat the steps above in a loop until the stopping criteria is met for the `should_stop` function.

In [51]:
# main function
def k_means(data, k):
    data = data.to_numpy()
    centroids = initialize_centroids(data, k)
    
    while True:
        old_centroids = centroids
        labels = get_labels(data, centroids)
        centroids = update_centroids(data, labels, k)
        
        if should_stop(old_centroids, centroids):
            break
    return labels

The goal of the `initialize_centroids` function is to initialize the centroids randomly within the region of all of the data points. We perform the steps as follows:
- Initiate the minimum and maximum values of the x and y coordinates ("x_min", "x_max", "y_min", and "y_max") respectively with infinity values.
- Loop through the data to get the actual minimum and maximum values of the x and y coordinates using the `min` and `max` functions.
- Create a list of "k" numbers of "centroids" and for each of the centroids, we sample the x and y coordinates randomly (using the next helper function: `random_sample`) within the minimum and maximum range of all points.

In [3]:
# helper function: initialize centroids
def initialize_centroids(data, k):
    x_min = y_min = float('inf')
    x_max = y_max = float('-inf')
    
    for point in data:
        x_min = min(point[0], x_min)
        x_max = max(point[0], x_max)
        y_min = min(point[1], y_min)
        y_max = max(point[1], y_max)
    centroids = []
    
    for i in range(k):
        centroids.append([random_sample(x_min, x_max), random_sample(y_min, y_max)])
    return centroids

In `random_sample`, we sample a random value between the low and high boundaries. We use the `random.random` function to sample a number uniformly distributed between 0 and 1. Then we scale the number so its value will be between low and high boundaries.

In [4]:
# helper function: random sample
def random_sample(low, high):
    return low + (high - low) * random.random()

We will use the `get_labels` function to get labels for each data point:
- Initiate an outer loop where it goes through all the data points.
- Initiate an inner loop where it goes through all the centroids.
- Set the first minimum distance ("min_dist") as infinity.
- Use `get_distance` to find the new distance between the data point and the centroid.
- Update the minimum distance and assign a label to the new centroid when we find a closer point.

In [5]:
# helper function: get labels
def get_labels(data, centroids):
    labels = []
    for point in data:
        min_dist = float('inf')
        label = None
        for i, centroid in enumerate(centroids):
            new_dist = get_distance(point, centroid)
            if min_dist > new_dist:
                min_dist = new_dist
                label = i
        labels.append(label)
    return labels

Implement a widely used Euclidean distant as a distance metric in the `get_distance` function.

In [6]:
# helper function: get distance
def get_distance(p1, p2):
    a = 0
    for n in range(len(p1)):
        a += (p1[n] - p2[n]) ** 2
    return (a) ** 0.5

In the `update_centroids` function, we update the centroids using the mean of the points that are assigned to each centroid:
- Initialize "k" number of "new_centroids" and "counts" of points that belong to each cluster.
- Loop through the data points to add the data vectors to the new centroids and increment the "counts" according to the label initiated in the step above.
- Loop through all the newly computed centroids to compute the average of x and y coordinates by dividing the sum data vectors by the "counts" to obtain the new centroids.

In [7]:
# helper function: update centroids
def update_centroids(data, labels, k):
    new_centroids = [[0, 0] for i in range(k)]
    counts = [0] * k
    
    for point, label in zip(data, labels):
        new_centroids[label][0] += point[0]
        new_centroids[label][1] += point[1]
        counts[label] += 1
        
    for i, (x, y) in enumerate(new_centroids):
        new_centroids[i] = (x / counts[i], y/ counts[i])
    return new_centroids

For the `should_stop` function, we decide the stopping criteria to exit the loop by comparing the total movement (the Euclidean distance of new and old centroids) across all centroids with a threshold ("1e-5"). The function returns true if the total movement is less than the threshold.

In [8]:
# helper function: should stop
def should_stop(old_centroids, new_centroids, threshold = 1e-5):
    total_movement = 0
    for old_point, new_point in zip(old_centroids, new_centroids):
        total_movement += get_distance(old_point, new_point)
    return total_movement < threshold

## Model Comparison

For this section, we will be using the popular Iris dataset to perform the comparison between our k-means model and the scikit-learn `KMeans` model. The metric we will use for our comparison will be the outputted label from both models. Because our model can only take in two-dimensional points data, we will use only two variables ("SepalLengthCm" and "SepalWidthCm") for the models to cluster the data. The metric we will use for our comparison will be the outputted labels from both models.

### Library & Data Preparation

In [9]:
# load library
import pandas as pd
from sklearn.cluster import KMeans

In [46]:
# load dataset
df = pd.read_csv('data/Iris_Dataset.csv')
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [47]:
# separate independent and dependent variables
X = df[["SepalLengthCm", "SepalWidthCm"]] # independent variables
y = df["Species"] # dependent variable

In [48]:
# normalize data
X_normalized = (X - X.min()) / (X.max() - X.min()) # min-max normalization
X_normalized.head()

,SepalLengthCm,SepalWidthCm
0,0.222222,0.625000
1,0.166667,0.416667
2,0.111111,0.500000
3,0.083333,0.458333
4,0.194444,0.666667


### Models Building

In [49]:
# sklearn k-means model
model_sklearn  = KMeans(n_clusters = 3).fit(X_normalized)

print("Original Labels of [3, 18, 135] Indexes: ", [y[i] for i in (3, 81, 135)])
print("K-mean Model's Labels of [3, 18, 135] Indexes: ", [model_sklearn.labels_[i] for i in (3, 81, 135)])

c:\Users\theon\Anaconda3\envs\mlalgorithm\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\theon\Anaconda3\envs\mlalgorithm\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Original Labels of [3, 18, 135] Indexes:  ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
K-mean Model's Labels of [3, 18, 135] Indexes:  [0, 1, 2]


In [56]:
# our k-means model
model_scratch = k_means(X_normalized, k = 3)

print("Original Labels of [3, 18, 135] Indexes: ", [y[i] for i in (3, 81, 135)])
print("K-mean Model's Labels of [3, 18, 135] Indexes: ", [model_scratch[i] for i in (3, 81, 135)])

Original Labels of [3, 18, 135] Indexes:  ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
K-mean Model's Labels of [3, 18, 135] Indexes:  [1, 2, 0]


As we can see from the results above, both models are able to identify the indexes belonging to different groups. Hence, we have successfully built the k-means model from scratch!